In [3]:
import tkinter as tk
from tkinter import ttk
import matplotlib
#matplotlib.use("TkAgg") #backend
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib import style
import matplotlib.colors
import matplotlib.pyplot as plt
import numpy as np

import dask.dataframe as dask
import pandas as pd
from plots import wavelength_to_rgb
import line_profiler
import atexit
profile = line_profiler.LineProfiler()
atexit.register(profile.print_stats)
import matplotlib.animation as animation


In [4]:

opt = 1

def changeSpectradisp(option):
    global opt
    if option == 0:
        opt = 0
    if option == 1:
        opt = 1
    if option == 2:
        opt = 2

LARGE_FONT = ("Verdana", 12)
NORM_FONT = ("Verdana", 10)
SMALL_FONT = ("Verdana", 8)

style.use("ggplot")

In [6]:
wavel_df = pd.read_csv('long_de_onda_1_tira.csv')
wavel_array = wavel_df.iloc[:, 0].values
inten_df = dask.read_csv('inten-50micr-*.csv',header=None)
RGB_up_df = pd.read_csv('RGB-colors-50micron-arriba.csv',header=None)
RGB_down_df = pd.read_csv('RGB-colors-50micron-abajo.csv',header=None)
RGB_frames = [RGB_down_df,RGB_up_df]
RGB_df = pd.concat(RGB_frames)

xy_pos_df = pd.read_csv('xy_positions_FULL50micron.csv',header=None)

R_array = RGB_df.iloc[:,0].values
G_array = RGB_df.iloc[:,1].values
B_array = RGB_df.iloc[:,2].values

R = R_array.reshape(492,260)
G = G_array.reshape(492,260)
B = B_array.reshape(492,260)

img = np.empty((492,260,3), dtype=np.uint8)

img[:,:,0] = R
img[:,:,1] = G
img[:,:,2] = B

In [12]:
wavel_array = wavel_df.iloc[:, 0].values

In [16]:
inten_df

Dask DataFrame Structure:
                    0        1        2        3        4        5        6        7        8        9        10       11       12       13       14       15       16       17       18       19       20       21       22       23       24       25       26       27       28       29       30       31       32       33       34       35       36       37       38       39       40       41       42       43       44       45       46       47       48       49       50       51       52       53       54       55       56       57       58       59       60       61       62       63       64       65       66       67       68       69       70       71       72       73       74       75       76       77       78       79       80       81       82       83       84       85       86       87       88       89       90       91       92       93       94       95       96       97       98       99       100      101      102      103      104      105      106      107      108      109      110      111      112      113      114      115      116      117      118      119      120      121      122      123      124      125      126      127      128      129      130      131      132      133      134      135      136      137      138      139      140      141      142      143      144      145      146      147      148      149      150      151      152      153      154      155      156      157      158      159      160      161      162      163      164      165      166      167      168      169      170      171      172      173      174      175      176      177      178      179      180      181      182      183      184      185      186      187      188      189      190      191      192      193      194      195      196      197      198      199      200      201      202      203      204      205      206      207      208      209      210      211      212      213      214      215      216      217      218      219      220      221      222      223      224      225      226      227      228      229      230      231      232      233      234      235      236      237      238      239      240      241      242      243      244      245      246      247      248      249      250      251      252      253      254      255      256      257      258      259      260      261      262      263      264      265      266      267      268      269      270      271      272      273      274      275      276      277      278      279      280      281      282      283      284      285      286      287      288      289      290      291      292      293      294      295      296      297      298      299      300      301      302      303      304      305      306      307      308      309      310      311      312      313      314      315      316      317      318      319      320      321      322      323      324      325      326      327      328      329      330      331      332      333      334      335      336      337      338      339      340      341      342      343      344      345      346      347      348      349      350      351      352      353      354      355      356      357      358      359      360      361      362      363      364      365      366      367      368      369      370      371      372      373      374      375      376      377      378      379      380      381      382      383      384      385      386      387      388      389      390      391      392      393      394      395      396      397      398      399      400      401      402      403      404      405      406      407      408      409      410      411      412      413      414      415      416      417      418      419      420      421      422      423      424      425      426      427      428      429      430      431      432      433      434      435      436      437      438      439

In [18]:
import dask.dataframe as dd
import pandas as pd
df = pd.DataFrame({"A": [1, 2, 3], "B": [3, 4, 5]},index=['a', 'b', 'c'])
ddf = dd.from_pandas(df, npartitions=2)
ddf.head()

C:\Users\juanr\Anaconda3\envs\spectral_gui\lib\site-packages\dask\dataframe\core.py:5736: UserWarning: Insufficient elements for `head`. 5 elements requested, only 3 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,A,B
a,1,3
b,2,4
c,3,5


In [22]:
ddf.loc[['b', 'c'], ['A']]

,A
npartitions=1,
b,int64
c,...


In [30]:
inten_df.columns[2]
#df[df.columns[-1]]

2

In [33]:
inten_df_selected = inten_df[(inten_df.columns[0] == 10)]
#inten_df_selected = inten_df[inten_df['0'] == val]
inten_df_selected.compute()

KeyError: False

In [10]:

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def popupmsg(msg):
    popup = tk.Tk()
    def leavemini():
        popup.destroy()
    popup.wm_title("!")
    label = ttk.Label(popup,text=msg,font = NORM_FONT)
    label.pack(side="top",fill="x",pady=10)
    B1 = ttk.Button(popup,text = "Ok", command = leavemini )
    B1.pack()
    popup.mainloop()


In [11]:


class SpectralGui(tk.Tk):

    def __init__(self,*args,**kwargs):

        tk.Tk.__init__(self, *args, **kwargs)
        tk.Tk.wm_title(self,"Spectral GUI")
        tk.Tk.iconbitmap(self,'c:/users/juanr/downloads/logo_lec.ico')
        tk.Tk.state(self,'zoomed')

        container = tk.Frame(self)
        container.pack(side="top", fill="both",expand=True)
        container.grid_rowconfigure(0,weight=1)
        container.grid_columnconfigure(0, weight=1)

        menubar = tk.Menu(container)
        filemenu = tk.Menu(menubar, tearoff = 0)
        filemenu.add_command(label = "Save settings",command = lambda: popupmsg("Not supported yet"))
        filemenu.add_separator() #agrega una barra separadora entre las opciones
        filemenu.add_command(label = "Exit",command = quit)
        menubar.add_cascade(label = "File", menu = filemenu)

        spectraChoice = tk.Menu(menubar,tearoff=0)
        spectraChoice.add_command(label="Fast&Simple Spectra",
                                  command= lambda: changeSpectradisp(0))
                                  # command= lambda: popupmsg("Fast&Simple Spectra"))
        spectraChoice.add_separator()
        spectraChoice.add_command(label="Fancy Spectra",
                                  command=lambda: changeSpectradisp(1))
                                  # command=lambda: popupmsg("Fancy Spectra"))
        spectraChoice.add_separator()
        spectraChoice.add_command(label="Click On Demand",
                                  command=lambda: changeSpectradisp(2))
        menubar.add_cascade(label= "Spectra subplot",menu=spectraChoice)

        dataSetMenu = tk.Menu(menubar, tearoff=0)
        dataSetMenu.add_command(label="Quartz-Tungsten", command=lambda: popupmsg("Not supported yet."))
        dataSetMenu.add_separator()
        dataSetMenu.add_command(label="NIR Source", command=lambda: popupmsg("Not supported yet."))
        dataSetMenu.add_separator()
        dataSetMenu.add_command(label="R LED", command=lambda: popupmsg("Not supported yet."))
        dataSetMenu.add_separator()
        dataSetMenu.add_command(label="G LED", command=lambda: popupmsg("Not supported yet."))
        dataSetMenu.add_separator()
        dataSetMenu.add_command(label="B LED", command=lambda: popupmsg("Not supported yet."))
        menubar.add_cascade(label="Light Source Datasets", menu=dataSetMenu)

        helpmenu = tk.Menu(menubar, tearoff=0)
        helpmenu.add_command(label="Contact us", command=lambda: popupmsg("      JRR, HG & LEC \n juanreto@gmail.com"))
        menubar.add_cascade(label="Help", menu=helpmenu)




        tk.Tk.config(self,menu = menubar)

        self.frames = {} #DICCIONARIO
        frame = SpectralPage(container,self)
        self.frames[SpectralPage] = frame
        frame.grid(row=0, column=0, sticky="nsew")
        self.show_frame(SpectralPage)

    def show_frame(self,cont):
        frame = self.frames[cont]
        frame.tkraise()


In [ ]:
class SpectralPage(tk.Frame):
    @profile
    def __init__(self,parent,controller):
        tk.Frame.__init__(self, parent)
        self.fig, (self.a0, self.a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [2.5, 1]})

        self.a0.imshow(img,interpolation='none', aspect='auto',origin= 'lower',extent=[0.0, 13000.0, 0, 24500.0])
        self.a0.set_ylabel('y [\u03bcm]')
        self.a0.set_xlabel('x [\u03bcm]')

        self.a1.set_ylabel('Intensity [a.u.]')
        self.a1.set_xlabel('Wavelength [nm]')
        self.fig.tight_layout()
        self.canvas = FigureCanvasTkAgg(self.fig, self)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        toolbar = NavigationToolbar2Tk(self.canvas, self)
        toolbar.update()
        self.canvas._tkcanvas.pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)
        self.ani = animation.FuncAnimation(self.fig,self.animate,interval=100)

    @profile
    def motion(self,event):
        x_move = event.xdata
        y_move = event.ydata
        if opt == 0 or opt == 1:

            if self.a0 == event.inaxes:
                # print("Moving through X-Y plot")
                # x, y = event.inaxes.transData.inverted().transform((event.x, event.y)) #transforma de coordenadas del Tkinker Canvas a coordenadas del matplotlib plot
                # print("Mouse position: (%s %s)" % (x, y))
                x_lst = find_nearest(xy_pos_df.iloc[:,0],x_move)
                y_lst = find_nearest(xy_pos_df.iloc[:,1], y_move)
                # print(x_lst,y_lst)
                mouse_pos = xy_pos_df[(xy_pos_df.iloc[:, 0] == x_lst) & (xy_pos_df.iloc[:, 1] == y_lst)].index.tolist()
                # print(mouse_pos)
                self.a1.clear()
                self.a1.set_ylabel('Intensity [a.u.]')
                self.a1.set_xlabel('Wavelength [nm]')
                if len(mouse_pos) == 0:
                    pass
                else:
                    if opt == 0:
                        inten_df[(df.A > 0) & (df.B <= 10)]
                        self.a1.plot(wavel_df.iloc[:, 0], inten_df.iloc[mouse_pos, :].transpose(), '*')
                    if opt == 1:
                        # spectrum = np.column_stack((wavel_array,np.transpose(inten_array[mouse_pos,:])))
                        # np.asarray(wavel_array, inten_file.iloc[mouse_pos,:].transpose())
                        # w,I = spectrum_subplot(spectrum)
                        clim = (350, 780)
                        norm = plt.Normalize(*clim)
                        wl = np.arange(clim[0], clim[1] + 1, 2)
                        colorlist = list(zip(norm(wl), [wavelength_to_rgb(w) for w in wl]))
                        spectralmap = matplotlib.colors.LinearSegmentedColormap.from_list("spectrum", colorlist)
                        mouse_pos = mouse_pos[0]
                        wavelengths = wavel_array #wavel_array[:, 0]
                        spectrum = np.transpose(inten_array[mouse_pos, :])
                        plt.plot(wavel_array, spectrum, color='darkred')

                        y = np.linspace(0, np.max(spectrum), 100)
                        X, Y = np.meshgrid(wavelengths, y)

                        extent = (np.min(wavelengths), np.max(wavelengths), np.min(y), np.max(y))

                        plt.imshow(X, clim=clim, extent=extent, cmap=spectralmap, aspect='auto')
                        plt.xlabel('Wavelength [nm]')
                        plt.ylabel('Intensity [a.u.]')

                        plt.fill_between(wavelengths, spectrum, np.max(spectrum), color='w')

            else:
                self.a1.clear()
                self.a1.set_ylabel('Intensity [a.u.]')
                self.a1.set_xlabel('Wavelength [nm]')
                return

    def on_click(self, event):
        if self.a0 == event.inaxes:
            x_click = event.xdata
            y_click = event.ydata
            if opt == 2:
                self.a1.clear()
                x_lst = find_nearest(xy_pos_df.iloc[:, 0], x_click)
                y_lst = find_nearest(xy_pos_df.iloc[:, 1], y_click)
                mouse_pos = xy_pos_df[(xy_pos_df.iloc[:, 0] == x_lst) & (xy_pos_df.iloc[:, 1] == y_lst)].index.tolist()
                self.a1.plot(wavel_df.iloc[:, 0], inten_df.iloc[mouse_pos, :].transpose(), '*')

    @profile
    def animate(self,interval):
        self.canvas.callbacks.connect('motion_notify_event',self.motion)
        self.canvas.mpl_connect('button_press_event', self.on_click)
        self.canvas.draw()